In [2]:
from typing import Callable
import numpy as np
import math


In [3]:
def simpsons(f: Callable[[float], float], i: int, a: float, b: float) -> float:
    xs = np.linspace(a, b, i, True, dtype = np.float64)
    A: list[np.float64] = []
    assert i >= 2
    for i in range(1, len(xs)):
        m = (xs[i] + xs[i-1])/2
        a = (1/6) * (f(xs[i]) + 4*f(m) + f(xs[i-1]))*(xs[i]-xs[i-1])
        A.append(a)
    return sum(A)

In [6]:
if __name__ == '__main__':
    f = lambda x: x**2
    print(simpsons(f, 2, -1, 1))
    f3 = lambda x: x**3 + x**2 + x + 1
    print(simpsons(f3, 5, -1, 2))
    f4 = lambda x: x**4
    print(simpsons(f4, 4, -1, 2))
    sinfun = lambda x: math.sin(x) + x**3
    print(simpsons(sinfun, 2, 0, math.pi/2))
    expfun = lambda x: math.exp(x)
    print(simpsons(expfun, 4, 0, 1))
    print(math.e - 1)

0.6666666666666666
11.25
6.625
2.5242969248984983
1.7182891699208316
1.718281828459045
